In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from time import strptime

### Twitter

Read in data export from twitter platform, wrangle, and export cleaned csv

In [2]:
# 1/31 - 2/13
twitter = pd.read_csv('twitter_weekly_pull.csv')
print(twitter.info())
twitter.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 40 columns):
Tweet id                        150 non-null float64
Tweet permalink                 150 non-null object
Tweet text                      150 non-null object
time                            150 non-null object
impressions                     150 non-null int64
engagements                     150 non-null int64
engagement rate                 150 non-null float64
retweets                        150 non-null int64
replies                         150 non-null int64
likes                           150 non-null int64
user profile clicks             150 non-null int64
url clicks                      150 non-null int64
hashtag clicks                  150 non-null int64
detail expands                  150 non-null int64
permalink clicks                150 non-null int64
app opens                       150 non-null int64
app installs                    150 non-null int64
follows              

,Tweet id,Tweet permalink,Tweet text,time,impressions,engagements,engagement rate,retweets,replies,likes,...,promoted hashtag clicks,promoted detail expands,promoted permalink clicks,promoted app opens,promoted app installs,promoted follows,promoted email tweet,promoted dial phone,promoted media views,promoted media engagements
0,1.095820e+18,https://twitter.com/Total_Bellas/status/109581...,Tag your brunch buddy! 🍹 (And take them to see...,2019-02-13 23:00 +0000,5588,344,0.061560,30,5,178,...,-,-,-,-,-,-,-,-,-,-
1,1.095770e+18,https://twitter.com/Total_Bellas/status/109577...,"Tag your #Galentine for this year, #BellaArmy!...",2019-02-13 20:00 +0000,9006,740,0.082167,51,10,362,...,-,-,-,-,-,-,-,-,-,-
2,1.095400e+18,https://twitter.com/Total_Bellas/status/109539...,@crzylikeafox86 Way to make #TotalBellas trivi...,2019-02-12 19:03 +0000,260,19,0.073077,0,2,2,...,-,-,-,-,-,-,-,-,-,-
3,1.095370e+18,https://twitter.com/Total_Bellas/status/109536...,What pose makes you feel strong &amp; beautifu...,2019-02-12 17:00 +0000,9196,944,0.102653,45,21,457,...,-,-,-,-,-,-,-,-,-,-
4,1.095040e+18,https://twitter.com/Total_Bellas/status/109503...,What scared the 💩 out of Nikki while she was t...,2019-02-11 19:20 +0000,5895,206,0.034945,12,27,74,...,-,-,-,-,-,-,-,-,-,-


In [3]:
# filter out unneeded columns
twitter = twitter.filter(['Tweet permalink', 'Tweet text', 'time', 'impressions', 'engagements',
                          'engagement rate', 'retweets', 'replies', 'likes', 'media views', 'media engagements'])
twitter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 11 columns):
Tweet permalink      150 non-null object
Tweet text           150 non-null object
time                 150 non-null object
impressions          150 non-null int64
engagements          150 non-null int64
engagement rate      150 non-null float64
retweets             150 non-null int64
replies              150 non-null int64
likes                150 non-null int64
media views          150 non-null int64
media engagements    150 non-null int64
dtypes: float64(1), int64(7), object(3)
memory usage: 13.0+ KB


In [5]:
## create new columns needed

#create 'post type' column indicating whether reply or original post
twitter['post type'] = twitter['Tweet text'].map(lambda x: "reply" if x[0] == '@' else "post")

# create 'vis eng' column calculating visible engagements by adding retweets, replies, likes
twitter['vis eng'] = twitter['retweets'] + twitter['replies'] + twitter['likes']

# create 'view Rate' and 'completion rate' columns by dividing media views and engagements by impressions, respectively
twitter['view rate'] = twitter['media views']/twitter['impressions']
twitter['completion rate'] = twitter['media engagements']/twitter['impressions']

# convert 'date' column to datetime
twitter['time'] = pd.to_datetime(twitter['time'])

# create 'in-show' column - 203 aired on 01/27/19 - 9PM EST, so tweets between GMT 1/28 2AM - 6AM are "in-show"
# 204 aired on 2/10/19 - 9PM EST, so tweets between GMT 2/11 2AM - 6AM are "in-show"
twitter['in-show'] = twitter['time'].map(lambda x: "in-show" if x >= pd.to_datetime(['2019-01-28 02:00:00']) and x <= pd.to_datetime(['2019-01-28 06:00:00']) else "in-show" if x >= pd.to_datetime(['2019-02-03 02:00:00']) and x <= pd.to_datetime(['2019-02-03 06:00:00']) else "not")

# create empty 'Votes' column
twitter['votes'] = np.nan

twitter['time'] = twitter['time'].dt.normalize()
print(twitter['time'].dtype)

#Phase = premiere/continuity/finale
twitter['phase'] = twitter['time'].map(lambda x: "premiere" if x >= pd.to_datetime(['2019-01-10 00:00:00']) and x <= pd.to_datetime(['2019-01-16 11:59:00']) else "continuity" if (x >= pd.to_datetime(['2019-01-17 00:00:00']) and x <= pd.to_datetime(['2019-01-31 00:00:00'])) or (x >= pd.to_datetime(['2019-02-07 00:00:00']) and x <= pd.to_datetime(['2019-03-21 00:00:00'])) else "finale" if x >= pd.to_datetime(['2019-03-21 00:00:00']) and x <= pd.to_datetime(['2019-03-28 00:00:00']) else "hiatus")

# add 'episode' column assigning episode number: first 2 digits = season number, last 2 digits = episode number
twitter['episode'] = twitter['time'].map(lambda x: "403" if x >= pd.to_datetime(['2019-01-24 00:00:00']) and x <= pd.to_datetime(['2019-01-30 11:59:00']) else "404" if x >= pd.to_datetime(['2019-02-07 00:00:00']) and x <= pd.to_datetime(['2019-02-13 11:59:00']) else "none")

#messaging = tune-in/awareness/catch-up
twitter['messaging'] = twitter['Tweet text'].map(lambda x: "tune-in" if 'Sunday' in x else "tune-in" if 'tomorrow' in x else "tune-in" if 'tonight' in x else "tune-in" if 'new' in x else "Catch-Up" if 'catch up' in x else "Catch-Up" if 'Demand' in x else "Awareness")

twitter['content series'] = np.nan

twitter['asset type'] = np.nan

twitter['subject'] = np.nan

print(twitter.info())
twitter.head()

datetime64[ns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 23 columns):
Tweet permalink      150 non-null object
Tweet text           150 non-null object
time                 150 non-null datetime64[ns]
impressions          150 non-null int64
engagements          150 non-null int64
engagement rate      150 non-null float64
retweets             150 non-null int64
replies              150 non-null int64
likes                150 non-null int64
media views          150 non-null int64
media engagements    150 non-null int64
post type            150 non-null object
vis eng              150 non-null int64
view rate            150 non-null float64
completion rate      150 non-null float64
in-show              150 non-null object
votes                0 non-null float64
phase                150 non-null object
episode              150 non-null object
messaging            150 non-null object
content series       0 non-null float64
asset type        

,Tweet permalink,Tweet text,time,impressions,engagements,engagement rate,retweets,replies,likes,media views,...,view rate,completion rate,in-show,votes,phase,episode,messaging,content series,asset type,subject
0,https://twitter.com/Total_Bellas/status/109581...,Tag your brunch buddy! 🍹 (And take them to see...,2019-02-13,5588,344,0.061560,30,5,178,1361,...,0.243558,0.020759,not,NaN,continuity,404,Awareness,NaN,NaN,NaN
1,https://twitter.com/Total_Bellas/status/109577...,"Tag your #Galentine for this year, #BellaArmy!...",2019-02-13,9006,740,0.082167,51,10,362,2006,...,0.222740,0.027981,not,NaN,continuity,404,Awareness,NaN,NaN,NaN
2,https://twitter.com/Total_Bellas/status/109539...,@crzylikeafox86 Way to make #TotalBellas trivi...,2019-02-12,260,19,0.073077,0,2,2,46,...,0.176923,0.030769,not,NaN,continuity,404,Awareness,NaN,NaN,NaN
3,https://twitter.com/Total_Bellas/status/109536...,What pose makes you feel strong &amp; beautifu...,2019-02-12,9196,944,0.102653,45,21,457,313,...,0.034037,0.034037,not,NaN,continuity,404,Awareness,NaN,NaN,NaN
4,https://twitter.com/Total_Bellas/status/109503...,What scared the 💩 out of Nikki while she was t...,2019-02-11,5895,206,0.034945,12,27,74,0,...,0.000000,0.000000,not,NaN,continuity,404,Awareness,NaN,NaN,NaN


In [6]:
#standardize column names i.e. make all lowercase
twitter.columns = map(str.lower, twitter.columns)
twitter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 23 columns):
tweet permalink      150 non-null object
tweet text           150 non-null object
time                 150 non-null datetime64[ns]
impressions          150 non-null int64
engagements          150 non-null int64
engagement rate      150 non-null float64
retweets             150 non-null int64
replies              150 non-null int64
likes                150 non-null int64
media views          150 non-null int64
media engagements    150 non-null int64
post type            150 non-null object
vis eng              150 non-null int64
view rate            150 non-null float64
completion rate      150 non-null float64
in-show              150 non-null object
votes                0 non-null float64
phase                150 non-null object
episode              150 non-null object
messaging            150 non-null object
content series       0 non-null float64
asset type           0 non-null f

In [7]:
#reorder columns
twitter = twitter[['post type', 'tweet permalink', 'tweet text', 'episode', 'time', 'in-show','phase','messaging',
                   'content series','asset type','subject','impressions','engagements', 'engagement rate', 'retweets', 
                   'replies', 'likes', 'votes', 'vis eng','media views', 'media engagements', 'view rate', 'completion rate']]
twitter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 23 columns):
post type            150 non-null object
tweet permalink      150 non-null object
tweet text           150 non-null object
episode              150 non-null object
time                 150 non-null datetime64[ns]
in-show              150 non-null object
phase                150 non-null object
messaging            150 non-null object
content series       0 non-null float64
asset type           0 non-null float64
subject              0 non-null float64
impressions          150 non-null int64
engagements          150 non-null int64
engagement rate      150 non-null float64
retweets             150 non-null int64
replies              150 non-null int64
likes                150 non-null int64
votes                0 non-null float64
vis eng              150 non-null int64
media views          150 non-null int64
media engagements    150 non-null int64
view rate            150 non-null flo

In [8]:
# convert new dataframe to csv "twitter_weekly.csv"
twitter.to_csv('twitter_weekly.csv')

### Nielsen
Read in data export from Nielsen UI, wrangle and write to csv.

In [24]:
# read in file for current week's episode
# air time: 2/10/19 - 9PM EST
nielsen = pd.read_csv('nielsen_weekly_pull.csv')
nielsen.info()
nielsen.head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2344 entries, 0 to 2343
Data columns (total 14 columns):
Social Content Ratings    2336 non-null object
Unnamed: 1                2334 non-null object
Unnamed: 2                2325 non-null object
Unnamed: 3                2325 non-null object
Unnamed: 4                2325 non-null object
Unnamed: 5                2321 non-null object
Unnamed: 6                2321 non-null object
Unnamed: 7                2319 non-null object
Unnamed: 8                2319 non-null object
Unnamed: 9                2319 non-null object
Unnamed: 10               2319 non-null object
Unnamed: 11               2319 non-null object
Unnamed: 12               2319 non-null object
Unnamed: 13               1927 non-null object
dtypes: object(14)
memory usage: 256.5+ KB


,Social Content Ratings,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,http://scr.nielsensocial.com/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Copyright 2019 The Nielsen Company. All right...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Network,E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Program,Total Bellas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Program Genre Type,Reality Series; Docs & Special Interests,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Season Number,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Episode,It's My Life,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Episode Number,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,EPG Program ID,SH024910700000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
# toss row indices 0-9 into a new dataframe and isolate episode name
episode_details = nielsen.iloc[:13]
episode_details['episode'] = episode_details.iat[6,1] + str('0') + episode_details.iat[8,1]
episode_details['name'] = episode_details.iat[7,1]
episode_details['air date'] = episode_details.iat[11,1]
episode_details['air time'] = episode_details.iat[12,1]
episode_details.drop(episode_details.index[1:],inplace=True)
episode_details = episode_details.loc[:, ['episode','name','air date','air time']]
episode_details

/Users/christina.caddel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/christina.caddel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/christina.caddel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

,episode,name,air date,air time
0,404,It's My Life,2/10/19,09:00 PM EST


In [26]:
# bring up row index 17 as header and drop row indices 0-16
nielsen.drop(nielsen.index[:17], inplace=True)
nielsen.columns = nielsen.iloc[0]
nielsen = nielsen[1:]
nielsen = nielsen.reset_index(drop = True)
nielsen.head(15)

17,nan,Unique Authors,Original Content,Unique Engagers,Total Engagement,Uniques,Interactions,nan,nan,nan,nan,nan,nan,nan
0,Twitter,234,392,4597,11531,4754,11923,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,Retweets,576,1284,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,Replies,389,529,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,Quotes,88,113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,Likes,4209,9605,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Tweets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,User,User Profile Url,User ID,Followers,Time Sent (Local),Local Date,Time Sent (GMT),Tweet ID,Tweet,Tweet URL,Interaction Type,Tweet Type,Root ID,Root URL
8,UsmanKhera1,https://twitter.com/UsmanKhera1,9.35895E+17,17,07:04:19 PM EST,2/10/19,2019-02-11T00:04:19Z,1.09475E+18,RT @BellaTwins: An all NEW episode of totalbel...,https://twitter.com/statuses/1094748932532592640,engagement,retweet,1.09464E+18,https://twitter.com/statuses/1094638464073515008
9,Awe_Heck,https://twitter.com/Awe_Heck,2972451204,20,07:46:12 PM EST,2/10/19,2019-02-11T00:46:12Z,1.09476E+18,@BellaTwins I can't wait! Oh and I got my Bell...,https://twitter.com/statuses/1094759470318653441,engagement,reply,1.09464E+18,https://twitter.com/statuses/1094638464073515008


In [27]:
# calculate value for total mentions and add as column
mentions = nielsen.iloc[8:]
nielsen['total mentions'] = mentions['Unique Authors'].count()

nielsen.head()

17,nan,Unique Authors,Original Content,Unique Engagers,Total Engagement,Uniques,Interactions,nan,nan,nan,nan,nan,nan,nan,total mentions
0,Twitter,234,392,4597,11531,4754,11923,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2318
1,NaN,NaN,Retweets,576,1284,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2318
2,NaN,NaN,Replies,389,529,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2318
3,NaN,NaN,Quotes,88,113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2318
4,NaN,NaN,Likes,4209,9605,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2318


In [28]:
# keep only relevant columns and drop all rows except first
nielsen = nielsen[['Unique Authors','Original Content','Unique Engagers','Interactions','total mentions']]
nielsen.drop(nielsen.index[1:], inplace=True)
nielsen.head()

17,Unique Authors,Original Content,Unique Engagers,Interactions,total mentions
0,234,392,4597,11923,2318


In [29]:
# rename and reorder columns
nielsen.columns = ['unique users','total social convos','unique authors/engagers','interactions','total mentions']
nielsen['episode'], nielsen['name'], nielsen['air date'], nielsen['air time'] = episode_details['episode'], episode_details['name'], episode_details['air date'], episode_details['air time']
nielsen = nielsen[['episode','name','air date','air time','unique users','total social convos','unique authors/engagers','interactions','total mentions']]
nielsen.head()

,episode,name,air date,air time,unique users,total social convos,unique authors/engagers,interactions,total mentions
0,404,It's My Life,2/10/19,09:00 PM EST,234,392,4597,11923,2318


In [30]:
nielsen.to_csv('nielsen_weekly.csv')

### Facebook
Read in data export from Facebook platform, wrangle, and write to csv.

In [16]:
fb = pd.read_csv('fb_weekly_pull.csv')
print(fb.info())
fb.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 54 columns):
Post ID                                                                        35 non-null object
Permalink                                                                      35 non-null object
Post Message                                                                   35 non-null object
Type                                                                           35 non-null object
Countries                                                                      0 non-null float64
Languages                                                                      0 non-null float64
Posted                                                                         35 non-null object
Audience Targeting                                                             35 non-null object
Lifetime Post Total Reach                                                      36 non-null object
Lifetime Post o

,Post ID,Permalink,Post Message,Type,Countries,Languages,Posted,Audience Targeting,Lifetime Post Total Reach,Lifetime Post organic reach,...,Lifetime Matched Audience Targeting Consumptions by Type - other clicks,Lifetime Matched Audience Targeting Consumptions by Type - link clicks,Lifetime Matched Audience Targeting Consumptions by Type - video play,Lifetime Matched Audience Targeting Consumptions by Type - photo view,Lifetime Negative Feedback from Users by Type - hide_all_clicks,Lifetime Negative Feedback from Users by Type - hide_clicks,Lifetime Negative Feedback from Users by Type - report_spam_clicks,Lifetime Negative Feedback by Type - hide_all_clicks,Lifetime Negative Feedback by Type - hide_clicks,Lifetime Negative Feedback by Type - report_spam_clicks
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lifetime: The number of people who had your Pa...,Lifetime: The number of people who had your Pa...,...,Lifetime: The number of clicks anywhere in the...,NaN,NaN,NaN,Lifetime: The number of times people have give...,NaN,NaN,Lifetime: The number of people who have given ...,NaN,NaN
1,1121717847895069_2277015872365255,https://www.facebook.com/totalbellas/posts/227...,"Parenting is a full-time job, and Brie is putt...",Link,NaN,NaN,2/13/19 18:13,,30392,30392,...,825,1130.0,NaN,NaN,3,4.0,NaN,3,4.0,NaN
2,1121717847895069_2266016210131888,https://www.facebook.com/totalbellas/posts/226...,A media frenzy won't scare Nikki. Brie on the ...,Video,NaN,NaN,2/12/19 9:00,,24411,24411,...,1706,26.0,511.0,NaN,4,3.0,NaN,3,3.0,NaN
3,1121717847895069_2273540462712796,https://www.facebook.com/totalbellas/posts/227...,"From the ring to the runway, the Bella Twins a...",Video,NaN,NaN,2/11/19 17:12,,28165,28165,...,2223,NaN,850.0,NaN,NaN,3.0,NaN,NaN,3.0,NaN
4,1121717847895069_2266013050132204,https://www.facebook.com/totalbellas/posts/226...,Brie’s forcing Nikki to draw a line in the san...,Video,NaN,NaN,2/11/19 9:00,,2566769,25106,...,7432,NaN,46917.0,NaN,8,5.0,1.0,8,5.0,1.0


In [17]:
#drop unwanted rows and columns
fb.drop(fb.index[0], inplace=True)
fb = fb.filter(['Permalink','Posted','Post Message','Type','Lifetime Post organic reach','Lifetime Post Organic Impressions','Lifetime Engaged Users','Lifetime Organic Video Views.1','Lifetime Organic views to 95%.1','Lifetime Talking About This (Post) by action type - like','Lifetime Talking About This (Post) by action type - comment','Lifetime Talking About This (Post) by action type - share'])
fb = fb.reset_index(drop=True)
fb.head()

,Permalink,Posted,Post Message,Type,Lifetime Post organic reach,Lifetime Post Organic Impressions,Lifetime Engaged Users,Lifetime Organic Video Views.1,Lifetime Organic views to 95%.1,Lifetime Talking About This (Post) by action type - like,Lifetime Talking About This (Post) by action type - comment,Lifetime Talking About This (Post) by action type - share
0,https://www.facebook.com/totalbellas/posts/227...,2/13/19 18:13,"Parenting is a full-time job, and Brie is putt...",Link,30392,39400,2190,0,0,668.0,16.0,19
1,https://www.facebook.com/totalbellas/posts/226...,2/12/19 9:00,A media frenzy won't scare Nikki. Brie on the ...,Video,24411,31636,2055,11059,1232,414.0,12.0,12
2,https://www.facebook.com/totalbellas/posts/227...,2/11/19 17:12,"From the ring to the runway, the Bella Twins a...",Video,28165,36936,2447,14166,1499,319.0,5.0,17
3,https://www.facebook.com/totalbellas/posts/226...,2/11/19 9:00,Brie’s forcing Nikki to draw a line in the san...,Video,25106,32214,45165,8862,3206,1298.0,89.0,83
4,https://www.facebook.com/totalbellas/posts/226...,2/10/19 17:00,Leave a 🙏 if Mama Bella's speaking the truth.\...,Video,48040,63224,6236,24182,12645,869.0,100.0,58


In [18]:
#rename columns
fb.columns = ['link','date posted','message','media type','reach','organic impressions','engagement','organic video views','organic video completes','likes','comments','shares']
fb.head()

,link,date posted,message,media type,reach,organic impressions,engagement,organic video views,organic video completes,likes,comments,shares
0,https://www.facebook.com/totalbellas/posts/227...,2/13/19 18:13,"Parenting is a full-time job, and Brie is putt...",Link,30392,39400,2190,0,0,668.0,16.0,19
1,https://www.facebook.com/totalbellas/posts/226...,2/12/19 9:00,A media frenzy won't scare Nikki. Brie on the ...,Video,24411,31636,2055,11059,1232,414.0,12.0,12
2,https://www.facebook.com/totalbellas/posts/227...,2/11/19 17:12,"From the ring to the runway, the Bella Twins a...",Video,28165,36936,2447,14166,1499,319.0,5.0,17
3,https://www.facebook.com/totalbellas/posts/226...,2/11/19 9:00,Brie’s forcing Nikki to draw a line in the san...,Video,25106,32214,45165,8862,3206,1298.0,89.0,83
4,https://www.facebook.com/totalbellas/posts/226...,2/10/19 17:00,Leave a 🙏 if Mama Bella's speaking the truth.\...,Video,48040,63224,6236,24182,12645,869.0,100.0,58


In [19]:
fb = fb.fillna(0)

cols = ['likes','comments','shares','reach','organic impressions','engagement','organic video views','organic video completes']

for col in cols:
    fb[col] = fb[col].astype(int)
    
fb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 12 columns):
link                       35 non-null object
date posted                35 non-null object
message                    35 non-null object
media type                 35 non-null object
reach                      35 non-null int64
organic impressions        35 non-null int64
engagement                 35 non-null int64
organic video views        35 non-null int64
organic video completes    35 non-null int64
likes                      35 non-null int64
comments                   35 non-null int64
shares                     35 non-null int64
dtypes: int64(8), object(4)
memory usage: 3.4+ KB


In [20]:
fb['date posted'] = pd.to_datetime(fb['date posted'])
fb['date posted'] = fb['date posted'].dt.normalize()
print(fb['date posted'].dtype)
fb.head()

datetime64[ns]


,link,date posted,message,media type,reach,organic impressions,engagement,organic video views,organic video completes,likes,comments,shares
0,https://www.facebook.com/totalbellas/posts/227...,2019-02-13,"Parenting is a full-time job, and Brie is putt...",Link,30392,39400,2190,0,0,668,16,19
1,https://www.facebook.com/totalbellas/posts/226...,2019-02-12,A media frenzy won't scare Nikki. Brie on the ...,Video,24411,31636,2055,11059,1232,414,12,12
2,https://www.facebook.com/totalbellas/posts/227...,2019-02-11,"From the ring to the runway, the Bella Twins a...",Video,28165,36936,2447,14166,1499,319,5,17
3,https://www.facebook.com/totalbellas/posts/226...,2019-02-11,Brie’s forcing Nikki to draw a line in the san...,Video,25106,32214,45165,8862,3206,1298,89,83
4,https://www.facebook.com/totalbellas/posts/226...,2019-02-10,Leave a 🙏 if Mama Bella's speaking the truth.\...,Video,48040,63224,6236,24182,12645,869,100,58


In [21]:
## Create and add needed columns
fb['vis eng'] = fb['shares'] + fb['likes'] + fb['comments']
fb['engagement rate'] = fb['engagement']/fb['reach']
fb['view rate'] = fb['organic video views']/fb['organic impressions']
fb['completion rate'] = fb['organic video completes']/fb['organic impressions']
fb['answers'] = np.nan

#Phase = pre-premiere/premiere/continuity/finale
fb['phase'] = fb['date posted'].map(lambda x: "premiere" if x >= pd.to_datetime(['2019-01-10 00:00:00']) and x <= pd.to_datetime(['2019-01-16 11:59:00']) else "continuity" if (x >= pd.to_datetime(['2019-01-17 00:00:00']) and x <= pd.to_datetime(['2019-01-31 00:00:00'])) or (x >= pd.to_datetime(['2019-02-07 00:00:00']) and x <= pd.to_datetime(['2019-03-21 00:00:00'])) else "finale" if x >= pd.to_datetime(['2019-03-21 00:00:00']) and x <= pd.to_datetime(['2019-03-28 00:00:00']) else "hiatus")

#messaging = tune-in/awareness/catch-up
fb['messaging'] = fb['message'].map(lambda x: "tune-in" if 'Sunday' in x else "tune-in" if 'tomorrow' in x else "tune-in" if 'tonight' in x else "tune-in" if 'new' in x else "Catch-Up" if 'catch up' in x else "Catch-Up" if 'Demand' in x else "Awareness")

fb['episode'] = fb['time'].map(lambda x: "403" if x >= pd.to_datetime(['2019-01-24 00:00:00']) and x <= pd.to_datetime(['2019-01-30 11:59:00']) else "404" if x >= pd.to_datetime(['2019-02-07 00:00:00']) and x <= pd.to_datetime(['2019-02-13 11:59:00']) else "none")

fb['content series'] = np.nan

fb['asset type'] = np.nan

fb['subject'] = np.nan

fb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 23 columns):
link                       35 non-null object
date posted                35 non-null datetime64[ns]
message                    35 non-null object
media type                 35 non-null object
reach                      35 non-null int64
organic impressions        35 non-null int64
engagement                 35 non-null int64
organic video views        35 non-null int64
organic video completes    35 non-null int64
likes                      35 non-null int64
comments                   35 non-null int64
shares                     35 non-null int64
vis eng                    35 non-null int64
engagement rate            35 non-null float64
view rate                  35 non-null float64
completion rate            35 non-null float64
answers                    0 non-null float64
phase                      35 non-null object
messaging                  35 non-null object
episode              

In [22]:
# reorder columns
fb = fb[['date posted','link','message','episode','media type','phase','messaging','content series','asset type','subject','reach','organic impressions','shares','likes','comments','answers','vis eng','engagement','engagement rate','organic video views','organic video completes','view rate','completion rate']]
fb.head()

,date posted,link,message,episode,media type,phase,messaging,content series,asset type,subject,...,likes,comments,answers,vis eng,engagement,engagement rate,organic video views,organic video completes,view rate,completion rate
0,2019-02-13,https://www.facebook.com/totalbellas/posts/227...,"Parenting is a full-time job, and Brie is putt...",404,Link,continuity,Awareness,NaN,NaN,NaN,...,668,16,NaN,703,2190,0.072058,0,0,0.000000,0.000000
1,2019-02-12,https://www.facebook.com/totalbellas/posts/226...,A media frenzy won't scare Nikki. Brie on the ...,404,Video,continuity,Catch-Up,NaN,NaN,NaN,...,414,12,NaN,438,2055,0.084183,11059,1232,0.349570,0.038943
2,2019-02-11,https://www.facebook.com/totalbellas/posts/227...,"From the ring to the runway, the Bella Twins a...",404,Video,continuity,Awareness,NaN,NaN,NaN,...,319,5,NaN,341,2447,0.086881,14166,1499,0.383528,0.040584
3,2019-02-11,https://www.facebook.com/totalbellas/posts/226...,Brie’s forcing Nikki to draw a line in the san...,404,Video,continuity,Catch-Up,NaN,NaN,NaN,...,1298,89,NaN,1470,45165,1.798972,8862,3206,0.275098,0.099522
4,2019-02-10,https://www.facebook.com/totalbellas/posts/226...,Leave a 🙏 if Mama Bella's speaking the truth.\...,404,Video,continuity,tune-in,NaN,NaN,NaN,...,869,100,NaN,1027,6236,0.129808,24182,12645,0.382481,0.200003


In [23]:
fb.to_csv('fb_weekly.csv')

### Instagram
Read in data export from Tracer, wrangle, and write to csv.

In [31]:
ig = pd.read_csv('ig_weekly_pull.csv')
print(ig.info())
ig.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 11 columns):
Page Name              22 non-null object
Post Created Date      22 non-null object
Post Permalink         21 non-null object
Post Media Type        21 non-null object
Post Caption           21 non-null object
Reach                  21 non-null float64
Impressions            21 non-null float64
Post Comments Count    21 non-null float64
Post Like Count        21 non-null float64
Engagement             21 non-null float64
Video Views            21 non-null float64
dtypes: float64(6), object(5)
memory usage: 2.1+ KB
None


,Page Name,Post Created Date,Post Permalink,Post Media Type,Post Caption,Reach,Impressions,Post Comments Count,Post Like Count,Engagement,Video Views
0,Total Bellas,1/24/19,https://www.instagram.com/p/BtBmix0hEo4/,VIDEO,Where’s the RT button? 🤔 Tag every member of t...,122610.0,150510.0,115.0,12937.0,13876.0,65116.0
1,Total Bellas,1/25/19,https://www.instagram.com/p/BtEL4GihjnT/,VIDEO,@thenikkibella’s enjoying her own brand of the...,91352.0,114871.0,82.0,8470.0,9044.0,45820.0
2,Total Bellas,1/27/19,https://www.instagram.com/p/BtJ9m-jhX6H/,IMAGE,Double-tap to call shotgun for eternity 😍 #Tot...,191644.0,231920.0,147.0,25957.0,26447.0,0.0
3,Total Bellas,1/28/19,https://www.instagram.com/p/BtKLVVvB_8E/,VIDEO,😉 if you'll be watching #TotalBellas with us t...,123998.0,154384.0,106.0,8735.0,9439.0,68080.0
4,Total Bellas,1/28/19,https://www.instagram.com/p/BtL5rDuh-J-/,IMAGE,"Once a wrestler, now a mom, forever @thebriebe...",177148.0,210366.0,97.0,21131.0,21540.0,0.0


In [32]:
#drop unwanted columns
ig.drop(columns = ['Page Name'], axis=1, inplace=True)
ig.drop(ig.index[-2:], inplace=True)
ig

,Post Created Date,Post Permalink,Post Media Type,Post Caption,Reach,Impressions,Post Comments Count,Post Like Count,Engagement,Video Views
0,1/24/19,https://www.instagram.com/p/BtBmix0hEo4/,VIDEO,Where’s the RT button? 🤔 Tag every member of t...,122610.0,150510.0,115.0,12937.0,13876.0,65116.0
1,1/25/19,https://www.instagram.com/p/BtEL4GihjnT/,VIDEO,@thenikkibella’s enjoying her own brand of the...,91352.0,114871.0,82.0,8470.0,9044.0,45820.0
2,1/27/19,https://www.instagram.com/p/BtJ9m-jhX6H/,IMAGE,Double-tap to call shotgun for eternity 😍 #Tot...,191644.0,231920.0,147.0,25957.0,26447.0,0.0
3,1/28/19,https://www.instagram.com/p/BtKLVVvB_8E/,VIDEO,😉 if you'll be watching #TotalBellas with us t...,123998.0,154384.0,106.0,8735.0,9439.0,68080.0
4,1/28/19,https://www.instagram.com/p/BtL5rDuh-J-/,IMAGE,"Once a wrestler, now a mom, forever @thebriebe...",177148.0,210366.0,97.0,21131.0,21540.0,0.0
5,1/29/19,https://www.instagram.com/p/BtOf9xKhBZ6/,IMAGE,Winston + @TheNikkiBella = our whole ❤ #TotalB...,97237.0,122468.0,46.0,11521.0,11678.0,0.0
6,1/31/19,https://www.instagram.com/p/BtR2NnWBWRW/,IMAGE,Our #wce 💜🔮 #TotalBellas,110974.0,138466.0,54.0,13559.0,13786.0,0.0
7,2/2/19,https://www.instagram.com/p/BtXAyEUBKNJ/,VIDEO,Can't a girl just live her life? 💁🏻#TotalBellas,84842.0,111631.0,93.0,7594.0,8118.0,51911.0
8,2/6/19,https://www.instagram.com/p/BtjU_hIB7Mf/,IMAGE,We are living for this Baby Bellas throwback! ...,139733.0,169668.0,96.0,17480.0,17799.0,0.0
9,2/7/19,https://www.instagram.com/p/Btl-NSQBTtf/,VIDEO,Tag someone that needs to see Nikki’s lunch da...,175452.0,227335.0,417.0,17657.0,18963.0,101374.0


In [33]:
#rename columns
ig.columns = ['date','link','media type','caption','reach','impressions','comments','likes','engagement','video views']

In [35]:
#add columns
ig['vis eng'] = ig['likes'] + ig['comments']
ig['eng rate'] = ig['engagement']/ig['impressions']
ig['view rate'] = ig['video views']/ig['impressions']

ig['date'] = pd.to_datetime(ig['date'])
ig['date'] = ig['date'].dt.normalize()

#Phase = premiere/continuity/finale
ig['phase'] = ig['date'].map(lambda x: "premiere" if x >= pd.to_datetime(['2019-01-10 00:00:00']) and x <= pd.to_datetime(['2019-01-16 11:59:00']) else "continuity" if (x >= pd.to_datetime(['2019-01-17 00:00:00']) and x <= pd.to_datetime(['2019-01-31 00:00:00'])) or (x >= pd.to_datetime(['2019-02-07 00:00:00']) and x <= pd.to_datetime(['2019-03-21 00:00:00'])) else "finale" if x >= pd.to_datetime(['2019-03-21 00:00:00']) and x <= pd.to_datetime(['2019-03-28 00:00:00']) else "hiatus")

#messaging column based on caption containing certain words - tune-in/awareness/catch-up
ig['messaging'] = ig['caption'].map(lambda x: "tune-in" if 'Sunday' in x else "tune-in" if 'tomorrow' in x else "tune-in" if 'tonight' in x else "tune-in" if 'new' in x else "Catch-Up" if 'catch up' in x else "Catch-Up" if 'Demand' in x else "Awareness")

ig['episode'] = ig['date'].map(lambda x: "403" if x >= pd.to_datetime(['2019-01-24 00:00:00']) and x <= pd.to_datetime(['2019-01-30 11:59:00']) else "404" if x >= pd.to_datetime(['2019-02-07 00:00:00']) and x <= pd.to_datetime(['2019-02-13 11:59:00']) else "none")

ig['content series'] = np.nan

ig['asset type'] = np.nan

ig['subject'] = np.nan

print(ig.info())
ig

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21 entries, 0 to 20
Data columns (total 19 columns):
date              21 non-null datetime64[ns]
link              21 non-null object
media type        21 non-null object
caption           21 non-null object
reach             21 non-null float64
impressions       21 non-null float64
comments          21 non-null float64
likes             21 non-null float64
engagement        21 non-null float64
video views       21 non-null float64
vis eng           21 non-null float64
eng rate          21 non-null float64
view rate         21 non-null float64
phase             21 non-null object
messaging         21 non-null object
episode           21 non-null object
content series    0 non-null float64
asset type        0 non-null float64
subject           0 non-null float64
dtypes: datetime64[ns](1), float64(12), object(6)
memory usage: 3.3+ KB
None


,date,link,media type,caption,reach,impressions,comments,likes,engagement,video views,vis eng,eng rate,view rate,phase,messaging,episode,content series,asset type,subject
0,2019-01-24,https://www.instagram.com/p/BtBmix0hEo4/,VIDEO,Where’s the RT button? 🤔 Tag every member of t...,122610.0,150510.0,115.0,12937.0,13876.0,65116.0,13052.0,0.092193,0.432636,continuity,tune-in,403,NaN,NaN,NaN
1,2019-01-25,https://www.instagram.com/p/BtEL4GihjnT/,VIDEO,@thenikkibella’s enjoying her own brand of the...,91352.0,114871.0,82.0,8470.0,9044.0,45820.0,8552.0,0.078732,0.398882,continuity,tune-in,403,NaN,NaN,NaN
2,2019-01-27,https://www.instagram.com/p/BtJ9m-jhX6H/,IMAGE,Double-tap to call shotgun for eternity 😍 #Tot...,191644.0,231920.0,147.0,25957.0,26447.0,0.0,26104.0,0.114035,0.000000,continuity,Awareness,403,NaN,NaN,NaN
3,2019-01-28,https://www.instagram.com/p/BtKLVVvB_8E/,VIDEO,😉 if you'll be watching #TotalBellas with us t...,123998.0,154384.0,106.0,8735.0,9439.0,68080.0,8841.0,0.061140,0.440978,continuity,tune-in,403,NaN,NaN,NaN
4,2019-01-28,https://www.instagram.com/p/BtL5rDuh-J-/,IMAGE,"Once a wrestler, now a mom, forever @thebriebe...",177148.0,210366.0,97.0,21131.0,21540.0,0.0,21228.0,0.102393,0.000000,continuity,Awareness,403,NaN,NaN,NaN
5,2019-01-29,https://www.instagram.com/p/BtOf9xKhBZ6/,IMAGE,Winston + @TheNikkiBella = our whole ❤ #TotalB...,97237.0,122468.0,46.0,11521.0,11678.0,0.0,11567.0,0.095356,0.000000,continuity,Awareness,403,NaN,NaN,NaN
6,2019-01-31,https://www.instagram.com/p/BtR2NnWBWRW/,IMAGE,Our #wce 💜🔮 #TotalBellas,110974.0,138466.0,54.0,13559.0,13786.0,0.0,13613.0,0.099562,0.000000,continuity,Awareness,none,NaN,NaN,NaN
7,2019-02-02,https://www.instagram.com/p/BtXAyEUBKNJ/,VIDEO,Can't a girl just live her life? 💁🏻#TotalBellas,84842.0,111631.0,93.0,7594.0,8118.0,51911.0,7687.0,0.072722,0.465023,hiatus,Awareness,none,NaN,NaN,NaN
8,2019-02-06,https://www.instagram.com/p/BtjU_hIB7Mf/,IMAGE,We are living for this Baby Bellas throwback! ...,139733.0,169668.0,96.0,17480.0,17799.0,0.0,17576.0,0.104905,0.000000,hiatus,Awareness,none,NaN,NaN,NaN
9,2019-02-07,https://www.instagram.com/p/Btl-NSQBTtf/,VIDEO,Tag someone that needs to see Nikki’s lunch da...,175452.0,227335.0,417.0,17657.0,18963.0,101374.0,18074.0,0.083414,0.445923,continuity,tune-in,404,NaN,NaN,NaN


In [36]:
#reorder columns
ig = ig[['date','link','media type','caption','episode','phase','messaging','content series','asset type','subject','reach','impressions','engagement','eng rate','likes','comments','vis eng','video views','view rate']]

ig.head()

,date,link,media type,caption,episode,phase,messaging,content series,asset type,subject,reach,impressions,engagement,eng rate,likes,comments,vis eng,video views,view rate
0,2019-01-24,https://www.instagram.com/p/BtBmix0hEo4/,VIDEO,Where’s the RT button? 🤔 Tag every member of t...,403,continuity,tune-in,NaN,NaN,NaN,122610.0,150510.0,13876.0,0.092193,12937.0,115.0,13052.0,65116.0,0.432636
1,2019-01-25,https://www.instagram.com/p/BtEL4GihjnT/,VIDEO,@thenikkibella’s enjoying her own brand of the...,403,continuity,tune-in,NaN,NaN,NaN,91352.0,114871.0,9044.0,0.078732,8470.0,82.0,8552.0,45820.0,0.398882
2,2019-01-27,https://www.instagram.com/p/BtJ9m-jhX6H/,IMAGE,Double-tap to call shotgun for eternity 😍 #Tot...,403,continuity,Awareness,NaN,NaN,NaN,191644.0,231920.0,26447.0,0.114035,25957.0,147.0,26104.0,0.0,0.000000
3,2019-01-28,https://www.instagram.com/p/BtKLVVvB_8E/,VIDEO,😉 if you'll be watching #TotalBellas with us t...,403,continuity,tune-in,NaN,NaN,NaN,123998.0,154384.0,9439.0,0.061140,8735.0,106.0,8841.0,68080.0,0.440978
4,2019-01-28,https://www.instagram.com/p/BtL5rDuh-J-/,IMAGE,"Once a wrestler, now a mom, forever @thebriebe...",403,continuity,Awareness,NaN,NaN,NaN,177148.0,210366.0,21540.0,0.102393,21131.0,97.0,21228.0,0.0,0.000000


In [37]:
#write to csv
ig.to_csv('ig_weekly.csv')